#  ICE_FLAVORS_RMV_VND_BRN_NAM_wip
   - module name: ICE_FLAVORS_RMV_VND_BRN_NAM_wip.ipnnb
   - purpose: Remove brand name from ECOMMERCE_DESCRIPTION
   - input: 
     - GTIN_NO and VND_BRN_NAM_102523.csv
     - GTIN_NO and VND_BRN_NAM_102523.csv
  
   - output:ICE_CREAM_FLAVORS_102523.xlsx  
   - functions: 
   - writen by: Sophia Yue
   - date: 10.25.23 

In [1]:
### Remove substrings like 'CHOCOLATE' if 'CHOCOLATE CHIP' is also present as a top result
def remove_substrings(string_list):
    str_df = pd.DataFrame(string_list, columns= ['Sentence'])
    str_df['Word Length'] = str_df.Sentence.apply(lambda x: len(x.split(' ')))
    str_df['Drop'] = ''
    str_df.sort_values('Word Length', inplace= True)
    str_df.reset_index(drop = True, inplace= True)
    for j in range(len(str_df)):
        str_1 = str_df.Sentence.loc[j]
        for i in range(len(str_df)-(j +1)):
            str_2 = str_df.Sentence.loc[i+j+1]
            if str_1 in str_2: 
                str_df.Drop.loc[j] = 'Yes'
                break
        if len(str_df) <2: 
            break
    return str_df[str_df.Drop != 'Yes'].Sentence.tolist()

In [2]:
#MAtch using 'token set ratio' with threshold
def flavor_match(item, flavor_list_org, cutoff = 75):
    #devowelized = [devowelize(i) for i in flavor_list_org]
    #derepeated = [remove_adjacent_repititions(i) for i in flavor_list_org]
    flavor_list = flavor_list_org #+ devowelized
    #flavor_list += derepeated
    #abbrev_maps = dict(zip(devowelized+ derepeated, flavor_list_org + flavor_list_org))
    #flavors_no_spaces = [i for i in flavor_list_org if not ' ' in i]
    #flavors_with_spaces = [i for i in Flavors_combo_bnj if ' ' in i]
    abbrev_maps = {}
    '''for i in flavors_no_spaces:
        for j in flavors_with_spaces:
            if j.replace(' ','') == i:
                abbrev_maps[i] = j'''
    abbrev_maps['NETFLIX']  = 'NETFLIX & CHILLED'   
    abbrev_maps['WAKE N BAKE'] = 'WAKE & NO BAKE'
    abbrev_maps['GIMMESMORE'] = "GIMME S'MORE!"
    abbrev_maps["GIMME S'MORE"] = "GIMME S'MORE!"
    abbrev_maps["COOKIES & CREAM"] = "COOKIES AND CREAM"
    abbrev_maps['MILK&COOKIES'] = 'MILK & COOKIES'
    # Add spaces to flavors as needed
    matched = process.extract(item, flavor_list, score_cutoff= cutoff, scorer = fuzz.token_set_ratio, processor=utils.default_process, limit = 7)
    flavor_shortlist = remove_substrings([i[0] for i in matched])
    matched = [i for i in matched if i[0] in flavor_shortlist]
    if len( flavor_shortlist) >0:
        max_score = matched[0][1]
        final_match = [ i for i in matched if i[1]== max_score]
        match_list = [i[0] for i in final_match]
        match_list = [abbrev_maps[i] if i in abbrev_maps.keys() else i for i in match_list]
        match_list = remove_substrings(match_list)
        # Second match using fuzz.partial_ratio
        rematched = process.extract(item, match_list,  scorer = fuzz.partial_ratio, processor=utils.default_process, limit = 5)
        re_max_score = rematched[0][1]
        final_rematch = [ i for i in rematched if i[1]== re_max_score]
        match_list =  [i[0] for i in final_rematch]
        '''
        match_list = [flavor_maps[i] if i in flavor_maps.keys() else i for i in match_list]
        if set(match_list) == {'STRAWBERRY','FRUIT'}: 
            match_list = ['FRUIT']#Drop Strawberry if fruit is the only other flavor
        if  set(match_list) != {'FRUIT'} : match_list = [ i for i in match_list if i != 'FRUIT']#Drop Fruit if it appears with something else
 
        if matched in multiflavor_dict.keys(): #Mapping multi-flavors to intended labels
            matched = multiflavor_dict[matched]
        '''
        flavor_count = len(set(match_list))
        matched = '|'.join(sorted(list(set(pd.Series(match_list)))))
        fuzz_score = max_score
    else: 
        matched = 'OTHER'
        flavor_count = 0
        fuzz_score = 0
    return (matched, flavor_count, fuzz_score)

In [121]:
def f_cr_vnd_map_dir():
    vnd_map = sorted(list(set(df_oth.VND_BRN_NAM.str.upper())))
    del vnd_map[0]
    vnd_map
    
    vnd_map_1 = [re.sub("'", '@', x) for x in vnd_map] # replace "'" with '@'
    vnd_map_1[0:20]
    
    strx = ": @@,"
    vnd_map_replace = [ "'" + x + "'" + strx for x in vnd_map_1]
    print(f'1. vnd_map_replace[0:20] = {vnd_map_replace[0:20]} /n  vnd_map_replace[-1] = {vnd_map_replace[-1] }')
    
    vnd_map_replace= ' '.join(vnd_map_replace) # convert a list to a string
    vnd_map_replace =vnd_map_replace.rstrip(',') # drop the last comma 
    print(f'2. vnd_map_replace[0:20] = {vnd_map_replace[0:20]} /n  vnd_map_replace[-1] = {vnd_map_replace[-1] }')
    
    vnd_map_replace = re.sub("'", '"', vnd_map_replace) 
    vnd_map_replace = re.sub('@', "'", vnd_map_replace) 
    vnd_map_replace = '{' + vnd_map_replace + '}'
    print(f'vnd_map_replace[0:20] = {vnd_map_replace[0:20] }/n  vnd_map_replace[-1] = {vnd_map_replace[-1] }')
    d_vnd_replace = eval(vnd_map_replace)
    rep = {"JOY® ":'',  "KROGER® ICE PIX ": '', 'KROGER® ' :'', '+DELX* ' :'', '+PRIS* ': '' }
    d_vnd_replace = {**rep, **d_vnd_replace}
    #print(f'3. d_vnd_replace[0:20] = {d_vnd_replace[0:20]}') 
    return d_vnd_replace

In [122]:
d_vnd_replace = f_cr_vnd_map_dir()

1. vnd_map_replace[0:20] = ["'AGGIE': @@,", "'AGLAMESIS BROTHERS': @@,", "'ALAMODE': @@,", "'ANDERSON DAIRY': @@,", "'ARCTIC ZERO': @@,", "'ATA RETAIL': @@,", "'BEN & JERRY@S': @@,", "'BFG': @@,", "'BLACK MARKET GELATO': @@,", "'BLUE BUNNY': @@,", "'BOLIS': @@,", "'BONBON GELATO & ICECREAM': @@,", "'BRAVE ROBOT': @@,", "'BREYERS': @@,", "'CADO': @@,", "'CARVEL': @@,", "'CEDAR CREST': @@,", "'CENTRAL DAIRY': @@,", "'CIAO BELLA GELATO COMPANY': @@,", "'COOL CLASSICS ICE PIX': @@,"] /n  vnd_map_replace[-1] = 'YASSO': @@,
2. vnd_map_replace[0:20] = 'AGGIE': @@, 'AGLAME /n  vnd_map_replace[-1] = @
vnd_map_replace[0:20] = {"AGGIE": '', "AGLAM/n  vnd_map_replace[-1] = }


In [123]:
d_vnd_replace

{'JOY® ': '',
 'KROGER® ICE PIX ': '',
 'KROGER® ': '',
 '+DELX* ': '',
 '+PRIS* ': '',
 'AGGIE': '',
 'AGLAMESIS BROTHERS': '',
 'ALAMODE': '',
 'ANDERSON DAIRY': '',
 'ARCTIC ZERO': '',
 'ATA RETAIL': '',
 "BEN & JERRY'S": '',
 'BFG': '',
 'BLACK MARKET GELATO': '',
 'BLUE BUNNY': '',
 'BOLIS': '',
 'BONBON GELATO & ICECREAM': '',
 'BRAVE ROBOT': '',
 'BREYERS': '',
 'CADO': '',
 'CARVEL': '',
 'CEDAR CREST': '',
 'CENTRAL DAIRY': '',
 'CIAO BELLA GELATO COMPANY': '',
 'COOL CLASSICS ICE PIX': '',
 'COUNTRY CHARM': '',
 'COUNTRY FRESH': '',
 'CRAFTSMAN': '',
 "DEAN'S": '',
 'DEANS': '',
 'DELUXE': '',
 'DENALI': '',
 'DENTONS': '',
 "DREYER'S/EDY'S": '',
 'EBERHARDS': '',
 'EDWARD & SONS': '',
 "EDY'S": '',
 'EL MEXICANO': '',
 'ENLIGHTENED': '',
 "F'REAL": '',
 'FARR': '',
 'FISHER': '',
 'FLAVOR ICE': '',
 'FLAVORICE': '',
 'FOODMARK': '',
 "FRIENDLY'S": '',
 'FROOZER': '',
 'GELATO PETRINI': '',
 'GOOD HUMOR': '',
 "GRAETER'S": '',
 'HAAGEN-DAZS': '',
 'HALO TOP': '',
 'HARRIS TEE

In [41]:
import pandas as pd, numpy as np, re
from datetime import date
from rapidfuzz import process, fuzz,utils
pd.options.display.float_format = '{:20,.2f}'.format 
pd.set_option("display.max_rows", None)
path = 'C:\\users\\iny2819\\Kroger\\Data\\' 
path_atb = path + 'ATB\\'
path_atb_yue = path + 'ATB\\atb_yue\\'

In [4]:
path_code = 'C:\\users\\iny2819\\kroger\\Code\\'  
f_com_code = path_code + "com_code.py"
exec(compile(open(f_com_code , "rb").read(), f_com_code, 'exec' ))

In [5]:
replacements = {'MOCHI': '',
 ' VAN ': ' VANILLA ',
 ' CHOC ': ' CHOCOLATE ',
   'CHUNCKY' :'CHUNKY',    
   'MARSCAPONE':'MASCARPONE',             
 'NEOPOLITAN': 'NEAPOLITAN',
 ' BDAY ': ' BIRTHDAY ',
 ' CKE ': ' CAKE ',
 ' CRML ': ' CARAMEL ',
   'VNL ': ' VANILLA ', 
   ' VNLLA ': ' VANILLA ',            
 "'N'": 'AND',
 "'N ": 'AND ',
 " N' ": ' AND ',               
 ' & CREAM': ' AND CREAM',
 ' MNG ': ' MANGO ',
 ' STRWBRY ': ' STRAWBERRY ',
 ' STRWBRRY': ' STRAWBERRY ' ,
 ' STRAWBERRIES': 'STRAWBERRY' , 
 ' STRWB ': ' STRAWBERRY ' ,  
  ' STRW ': ' STRAWBERRY ',
  ' BLBRY ' : ' BLUEBERRY',
   ' CKIE DGH ': ' COOKIE DOUGH ',         
' FRUITY ': ' FRUIT '  ,
' FRUITS ': ' FRUIT '  ,             
 ' PNUT ': ' PEANUT ',
 ' BTTR ': ' BUTTER ',
 ' PNUT BTR ': ' PEANUT BUTTER ',              
 'CRM ': 'CREAM ',
 'CREME': 'CREAM',              
 'CHP': 'CHIP',
 'MNT': 'MINT',
 ' MINT CHIP': ' MINT CHOCOLATE CHIP',  
  ' COOKIE N CRM': ' COOKIES AND CREAM',
  ' CKIE CRM ': ' COOKIES AND CREAM ',
  'CKIES&CRM'   :'COOKIES AN CREAM',          
  ' PNTBTTR' :'PEANUT BUTTER',
   'PNUT BT' : 'PEANUT BUTTER',   
    ' ALM ' : ' ALMOND ',   
  'ICE CREAM':'',   
   'DOUBLE RAINBOW':'',           
  ',': ' ',
   '/': ' ',
  '>':' ',
  '<': ' '}


In [10]:
df_vnd_brn_name = f_read(path, 'GTIN_NO and VND_BRN_NAM_102523.csv')
df_vnd_brn_name['GTIN_NO'] = df_vnd_brn_name['GTIN_NO'].astype(np.int64)


file GTIN_NO and VND_BRN_NAM_102523.csv shape (5219536, 2) 
 columns: Index(['GTIN_NO', 'VND_BRN_NAM'], dtype='object')
dtypes: 
 GTIN_NO        object
VND_BRN_NAM    object
dtype: object
head: 
           GTIN_NO VND_BRN_NAM
0  00000000000000    Optifast
1  00000000000017   KENNICOTT
2  00000000000024         NaN
3  00000000000031         NaN
4  00000000000130         NaN


In [19]:
df_oth = f_read(path_atb, 'ICE_CREAM_FLAVORS_102523__with_scores.xlsx', sheet_name = 'OTHERs')
df_oth['VND_ECOM_DSC'] = df_oth.VND_ECOM_DSC.fillna('')
df_oth.shape

file ICE_CREAM_FLAVORS_102523__with_scores.xlsx shape (881, 7) 
 columns: Index(['GTIN_NO', 'ECOMMERCE_DESCRIPTION', 'VND_ECOM_DSC', 'TAG_DESCRIPTION',
       'PRODUCT_DESCRIPTION/MKT_MSG', 'Flavor', 'Match_Score'],
      dtype='object')
dtypes: 
 GTIN_NO                         int64
ECOMMERCE_DESCRIPTION          object
VND_ECOM_DSC                   object
TAG_DESCRIPTION                object
PRODUCT_DESCRIPTION/MKT_MSG    object
Flavor                         object
Match_Score                     int64
dtype: object
head: 
           GTIN_NO                       ECOMMERCE_DESCRIPTION  \
0     76840003891               Ben & Jerry's Ice Cream Pints   
1     76840100552  Ben & Jerry's® Cherry Garcia Frozen Yogurt   
2  10076840003898                                      (null)   
3  10076840100559                                      (null)   
4           22231                                      (null)   

                                   VND_ECOM_DSC            TAG_DESCRIPTIO

(881, 7)

In [20]:

df_oth = pd.merge(df_oth,  df_vnd_brn_name, on = 'GTIN_NO', how ='left' )
df_oth.head()

,GTIN_NO,ECOMMERCE_DESCRIPTION,VND_ECOM_DSC,TAG_DESCRIPTION,PRODUCT_DESCRIPTION/MKT_MSG,Flavor,Match_Score,VND_BRN_NAM
0,76840003891,Ben & Jerry's Ice Cream Pints,Ben & Jerry's Ice Cream Mixed 1 PT,BNJR FAV CARVE ICE CREAM,<p>A euphoric lineup of Ben &amp; Jerry's top ...,OTHER,0,Ben & Jerry's
1,76840100552,Ben & Jerry's® Cherry Garcia Frozen Yogurt,Ben & Jerry's Ice Cream Frozen Yogurt 1 PT,BNJR CHRY GARCIA FRZ YGRT,<p>Ben &amp; Jerry's Cherry frozen yogurt with...,OTHER,0,Ben & Jerry's
2,10076840003898,(null),Ben & Jerry's Ice Cream Mixed 8 1 PT,(null),(null),OTHER,0,Ben & Jerry's
3,10076840100559,(null),Ben & Jerry's Ice Cream Frozen Yogurt 8 1 PT,(null),(null),OTHER,0,Ben & Jerry's
4,22231,(null),DELI GELATO SMALL,DELI GELATO SMALL,(null),OTHER,0,NaN


In [21]:
df_oth["ECOM_DSC_MOD"] = df_oth.ECOMMERCE_DESCRIPTION.fillna('').str.upper().replace(replacements, regex = True)
df_oth["ECOM_DSC_MOD"] = df_oth.ECOMMERCE_DESCRIPTION.fillna('').str.upper().replace(replacements, regex = True)
df_oth.head()

,GTIN_NO,ECOMMERCE_DESCRIPTION,VND_ECOM_DSC,TAG_DESCRIPTION,PRODUCT_DESCRIPTION/MKT_MSG,Flavor,Match_Score,VND_BRN_NAM,ECOM_DSC_MOD
0,76840003891,Ben & Jerry's Ice Cream Pints,Ben & Jerry's Ice Cream Mixed 1 PT,BNJR FAV CARVE ICE CREAM,<p>A euphoric lineup of Ben &amp; Jerry's top ...,OTHER,0,Ben & Jerry's,BEN & JERRY'S PINTS
1,76840100552,Ben & Jerry's® Cherry Garcia Frozen Yogurt,Ben & Jerry's Ice Cream Frozen Yogurt 1 PT,BNJR CHRY GARCIA FRZ YGRT,<p>Ben &amp; Jerry's Cherry frozen yogurt with...,OTHER,0,Ben & Jerry's,BEN & JERRY'S® CHERRY GARCIA FROZEN YOGURT
2,10076840003898,(null),Ben & Jerry's Ice Cream Mixed 8 1 PT,(null),(null),OTHER,0,Ben & Jerry's,(NULL)
3,10076840100559,(null),Ben & Jerry's Ice Cream Frozen Yogurt 8 1 PT,(null),(null),OTHER,0,Ben & Jerry's,(NULL)
4,22231,(null),DELI GELATO SMALL,DELI GELATO SMALL,(null),OTHER,0,NaN,(NULL)


In [15]:
l_vnd_brn_name = df_vnd_brn_name["VND_BRN_NAM"].nunique()  # int

In [16]:
l_vnd_brn_name = df_vnd_brn_name["VND_BRN_NAM"].unique()  # numpy.ndarray

In [18]:
df_vnd = pd.DataFrame(data = l_vnd_brn_name, columns = ['vnb_name'] )

In [20]:
df_vnd.to_excel(path_atb_yue + "vnd_brn_name.xlsx" )

In [21]:
df_vnd_brn_name["VND_BRN_NAM"].isnull().sum()

3821602

In [16]:


#df['col_3'] = df.apply(lambda x: f(x.col_1, x.col_2), axis=1)
#process.extract("barcelona", collection, scrr=fuzz.ratio))
#ic_df['pe_ratio_score']     = ic_df.apply(lambda x: process.extract(x.VND_ECOM_DSC, x.VND_BRN_NAM, scorer= fuzz.ratio), axis = 1)
ic_df['pe_p_ratio_score']   = ic_df.apply(lambda x: process.extract(x.VND_ECOM_DSC, x.VND_BRN_NAM, scorer= fuzz.partial_ratio), axis = 1)
#ic_df['pe_ts_ratio_score']  = ic_df.apply(lambda x: process.extract(x.VND_ECOM_DSC, x.VND_BRN_NAM, scorer= fuzz.token_sort_ratio), axis = 1)
#ic_df['pe_W_ratio_score']   = ic_df.apply(lambda x: process.extract(x.VND_ECOM_DSC, x.VND_BRN_NAM, scorer= fuzz.WRatio), axis = 1)

ic_df.head()

TypeError: 'float' object is not iterable

In [25]:
l_vnd_brn_name[0: 10]

NameError: name 'l_vnd_brn_name' is not defined

In [27]:

ic_df.to_excel(path_atb_yue + "ecom_desc_vnd_brn_name.xlsx" )

In [28]:
ic_df.dtypes

GTIN_NO         object
VND_ECOM_DSC    object
Flavor          object
VND_BRN_NAM     object
dtype: object

In [87]:
# NW AttributeError: 'Series' object has no attribute 'VND_ECOM_DSC'
from fuzzywuzzy import fuzz 
from fuzzywuzzy import process 
ic_df['pe_p_ratio_score']   = ic_df.apply(lambda x: process.extractBests(x.VND_ECOM_DSC, x.VND_BRN_NAM, scorer= fuzz.partial_ratio))

AttributeError: 'Series' object has no attribute 'VND_ECOM_DSC'

In [91]:
df_oth['ratio_score']    = ic_df.apply(lambda x:  fuzz.ratio(x.VND_ECOM_DSC, x.VND_BRN_NAM), axis = 1) 
df_oth['p_ratio_score']  = ic_df.apply(lambda x:  fuzz.partial_ratio(x.VND_ECOM_DSC, x.VND_BRN_NAM), axis = 1) 
df_oth['ts_ratio_score'] = ic_df.apply(lambda x:  fuzz.token_sort_ratio(x.VND_ECOM_DSC, x.VND_BRN_NAM), axis = 1) 
df_oth['W_rvatio_score']  = ic_df.apply(lambda x:  fuzz.WRatio(x.VND_ECOM_DSC, x.VND_BRN_NAM), axis = 1) 

TypeError: object of type 'float' has no len()

In [ ]:
process.extractOne
ic_df['pe.p_ratio_score']  = ic_df.apply(lambda x:  fuzz.partial_ratio(x.VND_ECOM_DSC, x.VND_BRN_NAM), axis = 1) 

In [34]:
ic_df.to_excel(path_atb_yue + "ecom_desc_vnd_brn_name_score.xlsx" )

In [56]:
df_vnd_brn_name.dtypes  

dtype('int64')

In [52]:
df_oth.dtypes

GTIN_NO                         int64
ECOMMERCE_DESCRIPTION          object
VND_ECOM_DSC                   object
TAG_DESCRIPTION                object
PRODUCT_DESCRIPTION/MKT_MSG    object
Flavor                         object
Match_Score                     int64
dtype: object

In [55]:
df_oth_1 = df_oth[df_oth.GTIN_NO.isin(df_vnd_brn_name.GTIN_NO)]
df_oth_1.shape 

 df_oth_res = df_oth_res[~df_oth_res.GTIN_NO.isin(blue_bunny.GTIN_NO)] 

AttributeError: 'Series' object has no attribute 'GTIN_NO'

In [69]:
df_oth.head()

,GTIN_NO,ECOMMERCE_DESCRIPTION,VND_ECOM_DSC,TAG_DESCRIPTION,PRODUCT_DESCRIPTION/MKT_MSG,Flavor,Match_Score,VND_BRN_NAM,ECOM_DSC_MOD
0,76840003891,Ben & Jerry's Ice Cream Pints,Ben & Jerry's Ice Cream Mixed 1 PT,BNJR FAV CARVE ICE CREAM,<p>A euphoric lineup of Ben &amp; Jerry's top ...,OTHER,0,Ben & Jerry's,BEN & JERRY'S PINTS
1,76840100552,Ben & Jerry's® Cherry Garcia Frozen Yogurt,Ben & Jerry's Ice Cream Frozen Yogurt 1 PT,BNJR CHRY GARCIA FRZ YGRT,<p>Ben &amp; Jerry's Cherry frozen yogurt with...,OTHER,0,Ben & Jerry's,BEN & JERRY'S® CHERRY GARCIA FROZEN YOGURT
2,10076840003898,(null),Ben & Jerry's Ice Cream Mixed 8 1 PT,(null),(null),OTHER,0,Ben & Jerry's,(NULL)
3,10076840100559,(null),Ben & Jerry's Ice Cream Frozen Yogurt 8 1 PT,(null),(null),OTHER,0,Ben & Jerry's,(NULL)
4,22231,(null),DELI GELATO SMALL,DELI GELATO SMALL,(null),OTHER,0,NaN,(NULL)


In [69]:
vnd_map = sorted(list(set(df_oth.VND_BRN_NAM)))
del vnd_map[0]
vnd_map

['AGGIE',
 'AGLAMESIS BROTHERS',
 'ARCTIC ZERO',
 'ATA RETAIL',
 'Alamode',
 'Anderson Dairy',
 'Arctic Zero',
 'BFG',
 'BLUE BUNNY',
 'BOLIS',
 'BONBON GELATO & ICECREAM',
 'BRAVE ROBOT',
 "Ben & Jerry's",
 'Black Market Gelato',
 'Blue Bunny',
 'Breyers',
 'CADO',
 'CARVEL',
 'CEDAR CREST',
 'CENTRAL DAIRY',
 'COOL CLASSICS ICE PIX',
 'COUNTRY FRESH',
 'CRAFTSMAN',
 'Ciao Bella Gelato Company',
 'Country Charm',
 'DEANS',
 'DELUXE',
 'DENALI',
 'DENTONS',
 "DREYER'S/EDY'S",
 "Dean's",
 'EBERHARDS',
 'EDWARD & SONS',
 "Edy's",
 'El Mexicano',
 'Enlightened',
 "F'REAL",
 'FISHER',
 'FLAVOR ICE',
 'FLAVORICE',
 'Farr',
 'Foodmark',
 "Friendly's",
 'Froozer',
 'GELATO PETRINI',
 'Good Humor',
 "Graeter's",
 'HAAGEN-DAZS',
 'HOMEMADE',
 'HT Traders',
 'Haagen-Dazs',
 'Halo Top',
 'Harris Teeter',
 'Helados Mexico',
 'Homemade Brand',
 'Howling Cow',
 'Hudsonville Ice Cream',
 'Icee',
 'JON DONAIRE',
 'JOY',
 "Jeni's Splendid Ice Creams",
 'Joy',
 "Julie's Organic Ice Cream",
 'KEMPS',
 'K

In [28]:
ex

C:\Users\iny2819\AppData\Local\Temp\ipykernel_2700\540123340.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  str_df.Drop.loc[j] = 'Yes'
C:\Users\iny2819\AppData\Local\Temp\ipykernel_2700\540123340.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  str_df.Drop.loc[j] = 'Yes'
C:\Users\iny2819\AppData\Local\Temp\ipykernel_2700\540123340.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  str_df.Drop.loc[j] = 'Yes'
C:\Users\iny2819

In [80]:
df_oth.head()

,GTIN_NO,ECOMMERCE_DESCRIPTION,VND_ECOM_DSC,TAG_DESCRIPTION,PRODUCT_DESCRIPTION/MKT_MSG,Flavor,Match_Score,VND_BRN_NAM,ECOM_DSC_MOD,vnd_mth
0,76840003891,Ben & Jerry's Ice Cream Pints,Ben & Jerry's Ice Cream Mixed 1 PT,BNJR FAV CARVE ICE CREAM,<p>A euphoric lineup of Ben &amp; Jerry's top ...,OTHER,0,Ben & Jerry's,BEN & JERRY'S PINTS,"(Ben & Jerry's, 1, 100.0)"
1,76840100552,Ben & Jerry's® Cherry Garcia Frozen Yogurt,Ben & Jerry's Ice Cream Frozen Yogurt 1 PT,BNJR CHRY GARCIA FRZ YGRT,<p>Ben &amp; Jerry's Cherry frozen yogurt with...,OTHER,0,Ben & Jerry's,BEN & JERRY'S® CHERRY GARCIA FROZEN YOGURT,"(Ben & Jerry's, 1, 100.0)"
2,10076840003898,(null),Ben & Jerry's Ice Cream Mixed 8 1 PT,(null),(null),OTHER,0,Ben & Jerry's,(NULL),"(OTHER, 0, 0)"
3,10076840100559,(null),Ben & Jerry's Ice Cream Frozen Yogurt 8 1 PT,(null),(null),OTHER,0,Ben & Jerry's,(NULL),"(OTHER, 0, 0)"
4,22231,(null),DELI GELATO SMALL,DELI GELATO SMALL,(null),OTHER,0,NaN,(NULL),"(OTHER, 0, 0)"


In [18]:
# NW: AttributeError: 'Series' object has no attribute 'ECOM_DSC_MOD'
#df_oth['vnd_mth1'] = df_oth.apply(lambda x:flavor_match(x.ECOM_DSC_MOD, flavor_list_org= x.VND_BRN_NAM,  cutoff = 75 ))
# NW KeyError: 'ECOM_DSC_MOD'
df_oth['vnd_mth1'] = df_oth.apply(lambda x:flavor_match(x['ECOM_DSC_MOD'], flavor_list_org= x['VND_BRN_NAM'],  cutoff = 75 ))

KeyError: 'ECOM_DSC_MOD'

In [89]:
#NW AttributeError: 'Series' object has no attribute 'VND_ECOM_DSC'
df_oth['vnd_mth1'] = df_oth.apply(lambda x:flavor_match(x.VND_ECOM_DSC, flavor_list_org= x.VND_BRN_NAM,  cutoff = 75 ))    

AttributeError: 'Series' object has no attribute 'VND_ECOM_DSC'

In [7]:
ic_df = f_read(path_atb, 'ICECREAM_FLAVORS_101223_pipe_delim.csv', sep = '|') # shape (11086, 3) 
ic_df['VND_ECOM_DSC'].isnull().sum()

file ICECREAM_FLAVORS_101223_pipe_delim.csv shape (11086, 3) 
 columns: Index(['GTIN_NO', 'VND_ECOM_DSC', 'Flavor'], dtype='object')
dtypes: 
 GTIN_NO         object
VND_ECOM_DSC    object
Flavor          object
dtype: object
head: 
           GTIN_NO               VND_ECOM_DSC  Flavor
0  00000000013772  COFFEE BAR - BREWED COFFE  COFFEE
1  00000000022231          DELI GELATO SMALL   OTHER
2  00000000022248         DELI GELATO MEDIUM   OTHER
3  00000000022255          DELI GELATO LARGE   OTHER
4  00000000022262           DELI GELATO PINT   OTHER


1

In [8]:
ic_df['VND_ECOM_DSC'] = ic_df.VND_ECOM_DSC.fillna('')

In [11]:
ic_df['GTIN_NO'] =ic_df['GTIN_NO'].astype(np.int64)
ic_df = pd.merge(ic_df,  df_vnd_brn_name, on = 'GTIN_NO', how ='left' )

In [12]:
ic_df['ratio_score']    = ic_df.apply(lambda x:  fuzz.ratio(x.VND_ECOM_DSC, x.VND_BRN_NAM), axis = 1) 
ic_df['p_ratio_score']  = ic_df.apply(lambda x:  fuzz.partial_ratio(x.VND_ECOM_DSC, x.VND_BRN_NAM), axis = 1) 
ic_df['ts_ratio_score'] = ic_df.apply(lambda x:  fuzz.token_sort_ratio(x.VND_ECOM_DSC, x.VND_BRN_NAM), axis = 1) 
ic_df['W_ratio_score']  = ic_df.apply(lambda x:  fuzz.WRatio(x.VND_ECOM_DSC, x.VND_BRN_NAM), axis = 1) 

In [82]:

df_oth.to_excel(path_atb_yue + "ecom_desc_vnd_brn_mth.xlsx" )

In [22]:
df_oth['ratio_score']    = df_oth.apply(lambda x:  fuzz.ratio(x.VND_ECOM_DSC, x.VND_BRN_NAM), axis = 1) 
df_oth['p_ratio_score']  = df_oth.apply(lambda x:  fuzz.partial_ratio(x.VND_ECOM_DSC, x.VND_BRN_NAM), axis = 1) 
df_oth['ts_ratio_score'] = df_oth.apply(lambda x:  fuzz.token_sort_ratio(x.VND_ECOM_DSC, x.VND_BRN_NAM), axis = 1) 
df_oth['W_ratio_score']  = df_oth.apply(lambda x:  fuzz.WRatio(x.VND_ECOM_DSC, x.VND_BRN_NAM), axis = 1) 

In [24]:

df_oth['ratio_score_m']    = df_oth.apply(lambda x:  fuzz.ratio(x.ECOM_DSC_MOD, x.VND_BRN_NAM), axis = 1) 
df_oth['p_ratio_score_m']  = df_oth.apply(lambda x:  fuzz.partial_ratio(x.ECOM_DSC_MOD, x.VND_BRN_NAM), axis = 1) 
df_oth['ts_ratio_score_m'] = df_oth.apply(lambda x:  fuzz.token_sort_ratio(x.ECOM_DSC_MOD, x.VND_BRN_NAM), axis = 1) 
df_oth['W_ratio_score_m']  = df_oth.apply(lambda x:  fuzz.WRatio(x.ECOM_DSC_MOD, x.VND_BRN_NAM), axis = 1)

In [35]:
df_oth['VND_BRN_NAM'] = df_oth.VND_BRN_NAM.fillna('')

In [47]:
for index, row in df_oth.iterrows(): 
    print (f' index = {index}')
    scr =  process.extract(row["VND_ECOM_DSC"], row["VND_BRN_NAM"].split(","), scorer= fuzz.partial_ratio)
    print(f'VND_ECOM_DSC {row["VND_ECOM_DSC"]}, VND_BRN_NAM= {row["VND_BRN_NAM"]}, scr = {scr}')

 index = 0
VND_ECOM_DSC Ben & Jerry's Ice Cream Mixed 1 PT, VND_BRN_NAM= Ben & Jerry's, scr = [("Ben & Jerry's", 100.0, 0)]
 index = 1
VND_ECOM_DSC Ben & Jerry's Ice Cream Frozen Yogurt 1 PT, VND_BRN_NAM= Ben & Jerry's, scr = [("Ben & Jerry's", 100.0, 0)]
 index = 2
VND_ECOM_DSC Ben & Jerry's Ice Cream Mixed 8 1 PT, VND_BRN_NAM= Ben & Jerry's, scr = [("Ben & Jerry's", 100.0, 0)]
 index = 3
VND_ECOM_DSC Ben & Jerry's Ice Cream Frozen Yogurt 8 1 PT, VND_BRN_NAM= Ben & Jerry's, scr = [("Ben & Jerry's", 100.0, 0)]
 index = 4
VND_ECOM_DSC DELI GELATO SMALL, VND_BRN_NAM= , scr = [('', 0.0, 0)]
 index = 5
VND_ECOM_DSC DELI GELATO MEDIUM, VND_BRN_NAM= , scr = [('', 0.0, 0)]
 index = 6
VND_ECOM_DSC DELI GELATO LARGE, VND_BRN_NAM= , scr = [('', 0.0, 0)]
 index = 7
VND_ECOM_DSC DELI GELATO PINT, VND_BRN_NAM= , scr = [('', 0.0, 0)]
 index = 8
VND_ECOM_DSC DELI GELATO QUART, VND_BRN_NAM= , scr = [('', 0.0, 0)]
 index = 9
VND_ECOM_DSC DELI SFT SRV ICE CRM CONE, VND_BRN_NAM= , scr = [('', 0.0, 0)]
 i

In [42]:
df_oth['pe_ratio_score']     = df_oth.apply(lambda x: process.extract(x.VND_ECOM_DSC, x.VND_BRN_NAM, scorer= fuzz.ratio), axis = 1)

In [43]:
df_oth.head()

,GTIN_NO,ECOMMERCE_DESCRIPTION,VND_ECOM_DSC,TAG_DESCRIPTION,PRODUCT_DESCRIPTION/MKT_MSG,Flavor,Match_Score,VND_BRN_NAM,ECOM_DSC_MOD,ratio_score,p_ratio_score,ts_ratio_score,W_ratio_score,ratio_score_m,p_ratio_score_m,ts_ratio_score_m,W_ratio_score_m,vnd_mth,pe_ratio_score
0,76840003891,Ben & Jerry's Ice Cream Pints,Ben & Jerry's Ice Cream Mixed 1 PT,BNJR FAV CARVE ICE CREAM,<p>A euphoric lineup of Ben &amp; Jerry's top ...,OTHER,0,Ben & Jerry's,BEN & JERRY'S PINTS,55.32,100.00,55.32,90.00,36.36,50.00,37.50,85.50,"(Ben & Jerry's, 1, 100.0)","[(B, 5.714285714285716, 0), (e, 5.714285714285..."
1,76840100552,Ben & Jerry's® Cherry Garcia Frozen Yogurt,Ben & Jerry's Ice Cream Frozen Yogurt 1 PT,BNJR CHRY GARCIA FRZ YGRT,<p>Ben &amp; Jerry's Cherry frozen yogurt with...,OTHER,0,Ben & Jerry's,BEN & JERRY'S® CHERRY GARCIA FROZEN YOGURT,47.27,100.00,47.27,90.00,21.82,50.00,21.82,85.50,"(Ben & Jerry's, 1, 100.0)","[(B, 4.651162790697672, 0), (e, 4.651162790697..."
2,10076840003898,(null),Ben & Jerry's Ice Cream Mixed 8 1 PT,(null),(null),OTHER,0,Ben & Jerry's,(NULL),53.06,100.00,53.06,90.00,0.00,0.00,0.00,0.00,"(OTHER, 0, 0)","[(B, 5.405405405405405, 0), (e, 5.405405405405..."
3,10076840100559,(null),Ben & Jerry's Ice Cream Frozen Yogurt 8 1 PT,(null),(null),OTHER,0,Ben & Jerry's,(NULL),45.61,100.00,45.61,90.00,0.00,0.00,0.00,0.00,"(OTHER, 0, 0)","[(B, 4.444444444444439, 0), (e, 4.444444444444..."
4,22231,(null),DELI GELATO SMALL,DELI GELATO SMALL,(null),OTHER,0,,(NULL),0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"(OTHER, 0, 0)",[]


In [49]:
df_oth_bkup = df_oth.copy()

In [92]:
vnd_map[0: 20]

['AGGIE',
 'AGLAMESIS BROTHERS',
 'ARCTIC ZERO',
 'ATA RETAIL',
 'Alamode',
 'Anderson Dairy',
 'Arctic Zero',
 'BFG',
 'BLUE BUNNY',
 'BOLIS',
 'BONBON GELATO & ICECREAM',
 'BRAVE ROBOT',
 "Ben & Jerry's",
 'Black Market Gelato',
 'Blue Bunny',
 'Breyers',
 'CADO',
 'CARVEL',
 'CEDAR CREST',
 'CENTRAL DAIRY']

In [95]:
vnd_map[0:20]

['AGGIE',
 'AGLAMESIS BROTHERS',
 'ARCTIC ZERO',
 'ATA RETAIL',
 'Alamode',
 'Anderson Dairy',
 'Arctic Zero',
 'BFG',
 'BLUE BUNNY',
 'BOLIS',
 'BONBON GELATO & ICECREAM',
 'BRAVE ROBOT',
 "Ben & Jerry's",
 'Black Market Gelato',
 'Blue Bunny',
 'Breyers',
 'CADO',
 'CARVEL',
 'CEDAR CREST',
 'CENTRAL DAIRY']

In [89]:
strx = ": '',"
vnd_map_replace = [ "'" + x + "'" + strx for x in vnd_map]
print(f'vnd_map_replace[0:20] = {vnd_map_replace[0:20] \n  vnd_map_replace[-1] = {vnd_map_replace[-1] }')


["'AGGIE': '',",
 "'AGLAMESIS BROTHERS': '',",
 "'ARCTIC ZERO': '',",
 "'ATA RETAIL': '',",
 "'Alamode': '',",
 "'Anderson Dairy': '',",
 "'Arctic Zero': '',",
 "'BFG': '',",
 "'BLUE BUNNY': '',",
 "'BOLIS': '',",
 "'BONBON GELATO & ICECREAM': '',",
 "'BRAVE ROBOT': '',",
 "'Ben & Jerry's': '',",
 "'Black Market Gelato': '',",
 "'Blue Bunny': '',",
 "'Breyers': '',",
 "'CADO': '',",
 "'CARVEL': '',",
 "'CEDAR CREST': '',",
 "'CENTRAL DAIRY': '',",
 "'COOL CLASSICS ICE PIX': '',",
 "'COUNTRY FRESH': '',",
 "'CRAFTSMAN': '',",
 "'Ciao Bella Gelato Company': '',",
 "'Country Charm': '',",
 "'DEANS': '',",
 "'DELUXE': '',",
 "'DENALI': '',",
 "'DENTONS': '',",
 "'DREYER'S/EDY'S': '',",
 "'Dean's': '',",
 "'EBERHARDS': '',",
 "'EDWARD & SONS': '',",
 "'Edy's': '',",
 "'El Mexicano': '',",
 "'Enlightened': '',",
 "'F'REAL': '',",
 "'FISHER': '',",
 "'FLAVOR ICE': '',",
 "'FLAVORICE': '',",
 "'Farr': '',",
 "'Foodmark': '',",
 "'Friendly's': '',",
 "'Froozer': '',",
 "'GELATO PETRINI': '',",


In [90]:
vnd_map_replace= ' '.join(vnd_map_replace)
vnd_map_replace

"'AGGIE': '', 'AGLAMESIS BROTHERS': '', 'ARCTIC ZERO': '', 'ATA RETAIL': '', 'Alamode': '', 'Anderson Dairy': '', 'Arctic Zero': '', 'BFG': '', 'BLUE BUNNY': '', 'BOLIS': '', 'BONBON GELATO & ICECREAM': '', 'BRAVE ROBOT': '', 'Ben & Jerry's': '', 'Black Market Gelato': '', 'Blue Bunny': '', 'Breyers': '', 'CADO': '', 'CARVEL': '', 'CEDAR CREST': '', 'CENTRAL DAIRY': '', 'COOL CLASSICS ICE PIX': '', 'COUNTRY FRESH': '', 'CRAFTSMAN': '', 'Ciao Bella Gelato Company': '', 'Country Charm': '', 'DEANS': '', 'DELUXE': '', 'DENALI': '', 'DENTONS': '', 'DREYER'S/EDY'S': '', 'Dean's': '', 'EBERHARDS': '', 'EDWARD & SONS': '', 'Edy's': '', 'El Mexicano': '', 'Enlightened': '', 'F'REAL': '', 'FISHER': '', 'FLAVOR ICE': '', 'FLAVORICE': '', 'Farr': '', 'Foodmark': '', 'Friendly's': '', 'Froozer': '', 'GELATO PETRINI': '', 'Good Humor': '', 'Graeter's': '', 'HAAGEN-DAZS': '', 'HOMEMADE': '', 'HT Traders': '', 'Haagen-Dazs': '', 'Halo Top': '', 'Harris Teeter': '', 'Helados Mexico': '', 'Homemade Bra

In [82]:
vnd_map_replace =vnd_map_replace.rstrip(',') 

In [ ]:
df_oth["ECOM_DSC_MOD"] = df_oth.ECOMMERCE_DESCRIPTION.fillna('').str.upper().replace(replacements, regex = True)
df_oth.head()

In [17]:
ic_df['ratio_score']    = ic_df.apply(lambda x:  fuzz.ratio(x.VND_ECOM_DSC, x.VND_BRN_NAM), axis = 1) 
ic_df['p_ratio_score']  = ic_df.apply(lambda x:  fuzz.partial_ratio(x.VND_ECOM_DSC, x.VND_BRN_NAM), axis = 1) 
ic_df['ts_ratio_score'] = ic_df.apply(lambda x:  fuzz.token_sort_ratio(x.VND_ECOM_DSC, x.VND_BRN_NAM), axis = 1) 
ic_df['W_rvatio_score']  = ic_df.apply(lambda x:  fuzz.WRatio(x.VND_ECOM_DSC, x.VND_BRN_NAM), axis = 1) 

In [83]:
vnd_map_replace = '{' + vnd_map_replace + '}'
vnd_map_replace

"{'AGGIE': '', 'AGLAMESIS BROTHERS': '', 'ARCTIC ZERO': '', 'ATA RETAIL': '', 'Alamode': '', 'Anderson Dairy': '', 'Arctic Zero': '', 'BFG': '', 'BLUE BUNNY': '', 'BOLIS': '', 'BONBON GELATO & ICECREAM': '', 'BRAVE ROBOT': '', 'Ben & Jerry's': '', 'Black Market Gelato': '', 'Blue Bunny': '', 'Breyers': '', 'CADO': '', 'CARVEL': '', 'CEDAR CREST': '', 'CENTRAL DAIRY': '', 'COOL CLASSICS ICE PIX': '', 'COUNTRY FRESH': '', 'CRAFTSMAN': '', 'Ciao Bella Gelato Company': '', 'Country Charm': '', 'DEANS': '', 'DELUXE': '', 'DENALI': '', 'DENTONS': '', 'DREYER'S/EDY'S': '', 'Dean's': '', 'EBERHARDS': '', 'EDWARD & SONS': '', 'Edy's': '', 'El Mexicano': '', 'Enlightened': '', 'F'REAL': '', 'FISHER': '', 'FLAVOR ICE': '', 'FLAVORICE': '', 'Farr': '', 'Foodmark': '', 'Friendly's': '', 'Froozer': '', 'GELATO PETRINI': '', 'Good Humor': '', 'Graeter's': '', 'HAAGEN-DAZS': '', 'HOMEMADE': '', 'HT Traders': '', 'Haagen-Dazs': '', 'Halo Top': '', 'Harris Teeter': '', 'Helados Mexico': '', 'Homemade Br

In [84]:
d_vnd = eval(vnd_map_replace) 
d_vnd

SyntaxError: invalid decimal literal (<string>, line 1)

In [62]:
vnd_map_replace =vnd_map_replace.rstrip(',') 

In [76]:
vnd_map_replace 

['AGGIE:,',
 'AGLAMESIS BROTHERS:,',
 'ARCTIC ZERO:,',
 'ATA RETAIL:,',
 'Alamode:,',
 'Anderson Dairy:,',
 'Arctic Zero:,',
 'BFG:,',
 'BLUE BUNNY:,',
 'BOLIS:,',
 'BONBON GELATO & ICECREAM:,',
 'BRAVE ROBOT:,',
 "Ben & Jerry's:,",
 'Black Market Gelato:,',
 'Blue Bunny:,',
 'Breyers:,',
 'CADO:,',
 'CARVEL:,',
 'CEDAR CREST:,',
 'CENTRAL DAIRY:,',
 'COOL CLASSICS ICE PIX:,',
 'COUNTRY FRESH:,',
 'CRAFTSMAN:,',
 'Ciao Bella Gelato Company:,',
 'Country Charm:,',
 'DEANS:,',
 'DELUXE:,',
 'DENALI:,',
 'DENTONS:,',
 "DREYER'S/EDY'S:,",
 "Dean's:,",
 'EBERHARDS:,',
 'EDWARD & SONS:,',
 "Edy's:,",
 'El Mexicano:,',
 'Enlightened:,',
 "F'REAL:,",
 'FISHER:,',
 'FLAVOR ICE:,',
 'FLAVORICE:,',
 'Farr:,',
 'Foodmark:,',
 "Friendly's:,",
 'Froozer:,',
 'GELATO PETRINI:,',
 'Good Humor:,',
 "Graeter's:,",
 'HAAGEN-DAZS:,',
 'HOMEMADE:,',
 'HT Traders:,',
 'Haagen-Dazs:,',
 'Halo Top:,',
 'Harris Teeter:,',
 'Helados Mexico:,',
 'Homemade Brand:,',
 'Howling Cow:,',
 'Hudsonville Ice Cream:,',
 

In [99]:
vnd_map_1 = [re.sub("'", '@', x) for x in vnd_map]
vnd_map_1[0:20]

['AGGIE',
 'AGLAMESIS BROTHERS',
 'ARCTIC ZERO',
 'ATA RETAIL',
 'Alamode',
 'Anderson Dairy',
 'Arctic Zero',
 'BFG',
 'BLUE BUNNY',
 'BOLIS',
 'BONBON GELATO & ICECREAM',
 'BRAVE ROBOT',
 'Ben & Jerry@s',
 'Black Market Gelato',
 'Blue Bunny',
 'Breyers',
 'CADO',
 'CARVEL',
 'CEDAR CREST',
 'CENTRAL DAIRY']

In [115]:
rep = {"Joy® ":'',  "Kroger® Ice Pix ": '', 'Kroger® ' :'' }

In [117]:
dic_vnd_map = {**d_vnd_replace, **rep}

In [118]:
dic_vnd_map

{'AGGIE': '',
 'AGLAMESIS BROTHERS': '',
 'ARCTIC ZERO': '',
 'ATA RETAIL': '',
 'Alamode': '',
 'Anderson Dairy': '',
 'Arctic Zero': '',
 'BFG': '',
 'BLUE BUNNY': '',
 'BOLIS': '',
 'BONBON GELATO & ICECREAM': '',
 'BRAVE ROBOT': '',
 "Ben & Jerry's": '',
 'Black Market Gelato': '',
 'Blue Bunny': '',
 'Breyers': '',
 'CADO': '',
 'CARVEL': '',
 'CEDAR CREST': '',
 'CENTRAL DAIRY': '',
 'COOL CLASSICS ICE PIX': '',
 'COUNTRY FRESH': '',
 'CRAFTSMAN': '',
 'Ciao Bella Gelato Company': '',
 'Country Charm': '',
 'DEANS': '',
 'DELUXE': '',
 'DENALI': '',
 'DENTONS': '',
 "DREYER'S/EDY'S": '',
 "Dean's": '',
 'EBERHARDS': '',
 'EDWARD & SONS': '',
 "Edy's": '',
 'El Mexicano': '',
 'Enlightened': '',
 "F'REAL": '',
 'FISHER': '',
 'FLAVOR ICE': '',
 'FLAVORICE': '',
 'Farr': '',
 'Foodmark': '',
 "Friendly's": '',
 'Froozer': '',
 'GELATO PETRINI': '',
 'Good Humor': '',
 "Graeter's": '',
 'HAAGEN-DAZS': '',
 'HOMEMADE': '',
 'HT Traders': '',
 'Haagen-Dazs': '',
 'Halo Top': '',
 'Har